In [15]:
import pandas as pd
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.models import TFTModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import rmse, mae, r2_score
from darts.utils.likelihood_models import QuantileRegression
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# ============================
# 1. 데이터 불러오기
df = pd.read_csv("total_data.csv")
df["Date"] = pd.to_datetime(df["Date"], format="%Y.%m.%d")
df = df.sort_values("Date").reset_index(drop=True)

# ============================
# 2. Business day 기준 리샘플링 
df = df.set_index("Date").asfreq("B")
df = df.ffill().bfill().reset_index()

# ============================
# 3. 상관분석 기반 변수 선택 + Tone_score 고정
corr = df.corr(numeric_only=True)["KOFR"].abs().sort_values(ascending=False)
top_vars = corr.index[1:11].tolist()
if "Tone_score" not in top_vars:
    top_vars.append("Tone_score")

df = df[["Date", "KOFR"] + top_vars]
print("최종 사용 변수:", df.columns.tolist())

# ============================
# 4. float32 변환
for col in df.select_dtypes(include=["float64"]).columns:
    df[col] = df[col].astype("float32")

# ============================
# 5. TimeSeries 변환
series = TimeSeries.from_dataframe(df, time_col="Date", value_cols="KOFR")
covariates = TimeSeries.from_dataframe(
    df,
    time_col="Date",
    value_cols=[c for c in df.columns if c not in ["Date", "KOFR"]]
)

# ============================
# 6. MinMax Scaling
scaler_y = Scaler()
scaler_x = Scaler()

series_scaled = scaler_y.fit_transform(series)
covariates_scaled = scaler_x.fit_transform(covariates)

train_series = series_scaled.slice(pd.Timestamp("2018-01-02"), pd.Timestamp("2024-12-31"))
test_series = series_scaled.slice(pd.Timestamp("2025-01-01"), pd.Timestamp("2025-07-31"))

train_cov = covariates_scaled.slice(pd.Timestamp("2018-01-02"), pd.Timestamp("2024-12-31"))
test_cov = covariates_scaled.slice(pd.Timestamp("2025-01-01"), pd.Timestamp("2025-07-31"))

최종 사용 변수: ['Date', 'KOFR', '콜금리(1일)', 'CD(91일)', 'KORIBOR(3개월)', '기준금리', 'EFFR', 'KORIBOR(6개월)', 'KORIBOR(12개월)', '경제활동참가율', '국고채(3년)', 'Core CPI', 'Tone_score']


In [16]:
# 7. TFT 모델 정의 (개선 세팅 적용)
early_stopper = EarlyStopping(monitor="val_loss", patience=10, min_delta=1e-4)

model = TFTModel(
    input_chunk_length=120,   # 입력 구간 확대
    output_chunk_length=7,    # horizon 줄임 (7일)
    hidden_size=64,           # 모델 복잡도 증가
    lstm_layers=2,
    dropout=0.2,
    batch_size=64,
    n_epochs=200,             # 학습 epoch 증가
    add_relative_index=True,
    likelihood=QuantileRegression(),
    random_state=42,
    pl_trainer_kwargs={
        "accelerator": "cpu",
        "precision": "32-true",
        "callbacks": [early_stopper]
    }
)

In [17]:
# 8. 모델 학습
model.fit(
    series=train_series,
    past_covariates=train_cov,
    val_series=test_series,
    val_past_covariates=test_cov,
    verbose=True
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/kook/myenv/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 24.0 K
5  | decoder_vsn                       | _VariableSelectionNetwork        | 1.6 K 
6  | sta

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

TFTModel(output_chunk_shift=0, hidden_size=64, lstm_layers=2, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.2, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=True, loss_fn=None, likelihood=QuantileRegression(quantiles=[0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99], prior_strength=1.0), norm_type=LayerNorm, use_static_covariates=True, input_chunk_length=120, output_chunk_length=7, batch_size=64, n_epochs=200, random_state=42, pl_trainer_kwargs={'accelerator': 'cpu', 'precision': '32-true', 'callbacks': [<pytorch_lightning.callbacks.early_stopping.EarlyStopping object at 0x331ecddc0>]})

In [18]:
# 9. 롤링 기반 예측 (7일 horizon)
pred_series = model.historical_forecasts(
    series_scaled,
    past_covariates=covariates_scaled,
    start=pd.Timestamp("2025-01-01"),
    forecast_horizon=7,
    stride=7,
    retrain=False,
    verbose=True
)

# 역스케일링
forecast = scaler_y.inverse_transform(pred_series)
test_series_inv = scaler_y.inverse_transform(test_series)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |                                             | 0/? [00:00<?, ?it/s]

In [ ]:
# 10. 성능 평가
print("RMSE:", rmse(test_series_inv, forecast))
print("MAE:", mae(test_series_inv, forecast))
print("R²:", r2_score(test_series_inv, forecast))

In [ ]:
# 11. 시각화
plt.figure(figsize=(10,5))
test_series_inv.plot(label="Actual (2025)")
forecast.plot(label="Predicted (2025)")
plt.title("KOFR Forecasting with TFT")
plt.legend()
plt.show()